In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import labolibrary as labo
import pickle
from sklearn.preprocessing import RobustScaler,PowerTransformer
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [10]:

#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'      

scalers = {}
# Function to center, scale, and return a series
def scale_group(group):
    scaler = RobustScaler()
    #scaler = PowerTransformer()
    scaled_values = scaler.fit_transform(group.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Store the scaler for this group
    return pd.Series(scaled_values, index=group.index, name=group.name)

# Function to inverse transform (de-scale) and decenter, and return a series
def inverse_scale_group(group):
    group_name = group.name
    scaler = scalers[group_name]
    inversed_centered_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    original_values = inversed_centered_values
    return pd.Series(original_values, index=group.index, name=group_name)


In [11]:

# Leer datos
#df_final = pd.read_parquet(DATOS_DIR+'FE_dataset-CARLA.parquet') 
#import dask.dataframe as dd
df_final = pd.read_parquet(DATOS_DIR+'FE_05_dataset.parquet') 
#df_final = df_final.compute()

df_final.columns = df_final.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)




In [12]:
## ESTO ES UN HACK PORQUE ESTA MAL ARMADA LA FUNCIÓN DE ESCALADO Y SOLO GUARDA LOS DATOS DE LA ÜLTIMA APLICACIÓN

df_final.groupby('product_id')['weight'].transform(scale_group)


periodo
2017-01     0.053000
2017-01     0.000000
2017-01    22.000175
2017-01     0.000000
2017-01     0.000000
             ...    
2019-12     0.000000
2019-12     0.000000
2019-12     0.000000
2019-12     0.000000
2019-12     5.026097
Freq: M, Name: weight, Length: 8042749, dtype: float64

In [13]:

### Filtrar datos
#df_true = df_final.loc['2019-12-01':'2020-01-01']
df_final = df_final.loc['2018-01-01':'2019-11-01']


In [14]:

#Filtro test
df_final = df_final[df_final['product_id'] < 20013]

weight= df_final[['weight','product_id','customer_id']]
#df_final.drop(columns=['weight'], inplace=True)
#filtered_weight = weight[weight['weight'] != 0]
#weight_mean = filtered_weight.groupby('product_id')['weight'].mean().to_dict()
#weight['weight'] = weight['product_id'].map(weight_mean)

weight = weight.groupby(['product_id','customer_id'])['weight'].mean()
weight = weight.reset_index()
df_final = df_final.drop(columns=['weight'])

In [16]:
# Correr Modelo
params={
        'boosting_type': 'gbdt',
        'objective': 'Regression',
        'metric':'rmse',
        'verbose': -1,
        #'n_jobs': -1,
        'seed': 10000079,
        #'learning_rate': 0.2,
        #'bagging_fraction': 0.85,
        #'bagging_freq': 1, 
        #'colsample_bytree': 0.85,
        #'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        #'lambda_l1': 0.5,
        #'lambda_l2': 0.5
}

predictions_all = pd.DataFrame(columns=['tn'])
products = df_final['product_id'].unique()
tot = len(products)
nro = 0
for producto in products:
    print(f'Fitting and predicting for product_id: {producto}')
    # Filtrar los datos del producto
    df_producto = df_final[df_final['product_id'] == producto]
    
    product_weights = pd.merge(df_producto.reset_index(), weight, on=['product_id', 'customer_id'], how='left')
    product_weights.set_index('periodo', inplace=True)
    weight_p = product_weights['weight']
   
    
    # Prepare data for LSTM on tn_2 only
    X = df_producto[['tn_2']].values.astype('float32')
    y = df_producto['tn_2'].values.astype('float32')
    X = X.reshape((X.shape[0], 1, X.shape[1]))
    #######################################################    
    # Define LSTM model
    model = Sequential()
    model.add(LSTM(50, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy')

    # Train LSTM model
    model.fit(X, y, epochs=10, batch_size=32, verbose=1)

    # Extract features from LSTM
    features = model.predict(X)

    # Prepare data for LightGBM
    # Convert LSTM features to DataFrame
    features_df = pd.DataFrame(features, index=df_producto.index, columns=[f'lstm_feature_{i}' for i in range(features.shape[1])])
    df_producto = pd.concat([df_producto, features_df], axis=1)
    df_producto['tn_2'] = y
    
    #############################################################################  
    #### Agrupar y escalar
   
    model, average_metric = labo.train_lightgbm_model_classic(df_producto,params,metric='rmse',
                                                      weights=weight_p)
    print("Overall rmse metric: ", average_metric)
    # Predict values for the entire dataset using the trained models
    # Prepare last data points for prediction
    last_data_points = df_producto[df_producto.index == df_producto.index.max()].copy()
    last_data_points.drop(columns=['tn_2'], inplace=True)
    # Predict the next month's value using the trained model
    predictions = labo.predict_next_month(model, last_data_points)
    preds = predictions.groupby('product_id')['tn_2'].transform(inverse_scale_group)
    predictions['tn'] = preds
    predictions.drop(columns=['tn_2'], inplace=True)
    predictions = predictions.reset_index()
    predictions =  predictions.groupby('product_id')['tn'].sum()
    predictions.columns = ['product_id', 'tn']
    predictions_all = pd.concat([predictions_all, predictions])
    print(predictions_all[-1:])


Fitting and predicting for product_id: 20011
Epoch 1/10


d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 731us/step - loss: -25.7842
Epoch 2/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step - loss: -104.2087
Epoch 3/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - loss: -176.2365
Epoch 4/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step - loss: -230.4949
Epoch 5/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 841us/step - loss: -286.0197
Epoch 6/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 782us/step - loss: -367.5173
Epoch 7/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step - loss: -419.9748
Epoch 8/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - loss: -485.0056
Epoch 9/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 796us/step - loss: -451.4237
Epoch 10/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - loss: -579.6445
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 903us/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[62]	validation's rmse: 19.8035
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[42]	validation's rmse: 

C:\Users\Admin\AppData\Local\Temp\ipykernel_16240\1599199677.py:75: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
C:\Users\Admin\AppData\Local\Temp\ipykernel_16240\1599199677.py:75: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using S

347/347 ━━━━━━━━━━━━━━━━━━━━ 1s 749us/step - loss: -14.8786
Epoch 2/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step - loss: -74.7192
Epoch 3/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step - loss: -108.1115
Epoch 4/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step - loss: -159.2943
Epoch 5/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - loss: -209.9376
Epoch 6/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 811us/step - loss: -253.7548
Epoch 7/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 939us/step - loss: -287.0210
Epoch 8/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - loss: -308.6546
Epoch 9/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 788us/step - loss: -359.5329
Epoch 10/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 781us/step - loss: -417.4660
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[26]	validation's rmse: 11.3789
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[19]	validation's rmse: 3

...

337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 692us/step - loss: -10.9927
Epoch 2/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - loss: -67.9207
Epoch 3/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - loss: -128.8891
Epoch 4/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step - loss: -187.4978
Epoch 5/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - loss: -213.1989
Epoch 6/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step - loss: -240.5724
Epoch 7/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step - loss: -297.7700
Epoch 8/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step - loss: -327.4114
Epoch 9/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step - loss: -367.4518
Epoch 10/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - loss: -375.7196
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[20]	validation's rmse: 11.7067
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[15]	validation's rmse: 1

d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 815us/step - loss: -33.9155
Epoch 2/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 786us/step - loss: -200.6044
Epoch 3/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step - loss: -319.1612
Epoch 4/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - loss: -399.9594
Epoch 5/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step - loss: -508.0833
Epoch 6/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - loss: -547.2249
Epoch 7/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step - loss: -660.3026
Epoch 8/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step - loss: -717.4920
Epoch 9/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step - loss: -764.4521
Epoch 10/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step - loss: -1061.0431
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[117]	validation's rmse: 33.633
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[36]	validation's rmse:

d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 713us/step - loss: -10.3790
Epoch 2/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 685us/step - loss: -58.8711
Epoch 3/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 691us/step - loss: -97.1576
Epoch 4/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step - loss: -127.9108
Epoch 5/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - loss: -142.1311
Epoch 6/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step - loss: -194.7042
Epoch 7/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 729us/step - loss: -229.8765
Epoch 8/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - loss: -235.0952
Epoch 9/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step - loss: -289.5558
Epoch 10/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 757us/step - loss: -280.3916
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[39]	validation's rmse: 9.13007
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[21]	validation's rmse: 6.

In [ ]:

predictions_all['tn']=predictions_all['tn'].astype('float32')
predictions_all.index.names = ['product_id']
predictions_all.to_csv(DATOS_DIR+'/pred/0025-prediccion-RSME_scaled_Robust-product_id_LSTM-weights2.csv', index=True,header=True)
print("Overall custom metric: ", average_metric)


Overall custom metric:  18.059966636174153
